In [1]:
# modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import chardet
import csv
import os
import matplotlib.dates as mdates
from IPython.display import Image

## Inserez les chemins vers le fichiers CSV:

In [9]:
# on récupère les informations du dossier
dossier = './raw_data'
contenuDossier = os.listdir(dossier)
print(contenuDossier)
nombrePoints = 0
nomPoints = []
numeroPoints = []
for x in contenuDossier:
    if x.startswith('point'):
        nombrePoints += 1
        nomPoints.append(x)
        numeroPoints.append(x[5:7])
print(numeroPoints)

['point13_29_07_15', 'point14_29_07_15', 'point15_29_07_15', 'schema_dispositif_AvAv_23_07_15.png']
['13', '14', '15']


In [25]:
def read_csv (chemin_fichier):
    #Detecter separateur
    with open(chemin_fichier, 'r') as file:
        sniffer = csv.Sniffer()
        sample_data = file.read(1024)
        detecter_separateur = (sniffer.sniff(sample_data).delimiter)

    if "Titre" in open(chemin_fichier).readline():
        data_frame = pd.read_csv(chemin_fichier, sep=detecter_separateur, skiprows=1)
    else:
        data_frame = pd.read_csv(chemin_fichier, sep=detecter_separateur)

    return data_frame 

In [26]:
# on crée un dictionnaire avec toutes les informations pour chaque point et on met les dictionnaires
# dans une liste
data = []
for x in nomPoints :
    dico = {}
    dico['numero'] = x[5:7]
    dico['nom'] = x
    dico['chemin'] = dossier + '/' + x
    with open(dico['chemin'] + '/geometrie.txt', 'r') as fichier:
        lignes = fichier.readlines()
        dico['capteur'] = lignes[1][:4]
        if lignes[3] == '\n' :
            dico['profondeur'] = 0
        else : 
            dico['profondeur'] = lignes[3]
        dico['profondeurMesures'] = lignes[5]
    contenuDossierPoint = os.listdir(dico['chemin'])
    for x in contenuDossierPoint:
        if x.startswith('p') and x.endswith('.csv'):
            dico['pression'] = read_csv(dico['chemin'] + '/' + x)
        if x.startswith('t') and x.endswith('.csv'):
            dico['temperature'] = read_csv(dico['chemin'] + '/' + x)
    data.append(dico)


In [27]:
# en faisant data[0] on a accès à toutes les informations du point 1
data[0]

{'numero': '13',
 'nom': 'point13_29_07_15',
 'chemin': './raw_data/point13_29_07_15',
 'capteur': 'p505',
 'profondeur': 0,
 'profondeurMesures': '10;20;30;40',
 'pression':        # Date Heure, GMT+02:00  \
 0      1  07/23/15 02:00:00 PM   
 1      2  07/23/15 02:15:00 PM   
 2      3  07/23/15 02:30:00 PM   
 3      4  07/23/15 02:45:00 PM   
 4      5  07/23/15 03:00:00 PM   
 ..   ...                   ...   
 558  559  07/29/15 09:30:00 AM   
 559  560  07/29/15 09:45:00 AM   
 560  561  07/29/15 10:00:00 AM   
 561  562  07/29/15 10:15:00 AM   
 562  563  07/29/15 10:30:00 AM   
 
      Tension, V (LGR S/N: 10117169, SEN S/N: 10117169)  \
 0                                              0.58181   
 1                                              0.58059   
 2                                              0.57937   
 3                                              0.57937   
 4                                              0.57937   
 ..                                               

In [28]:
# notamment la température et la pression
data[0]['pression']

,#,"Date Heure, GMT+02:00","Tension, V (LGR S/N: 10117169, SEN S/N: 10117169)","Temp., °C (LGR S/N: 10117169, SEN S/N: 10117169)"
0,1,07/23/15 02:00:00 PM,0.58181,18.129
1,2,07/23/15 02:15:00 PM,0.58059,18.081
2,3,07/23/15 02:30:00 PM,0.57937,18.033
3,4,07/23/15 02:45:00 PM,0.57937,18.224
4,5,07/23/15 03:00:00 PM,0.57937,18.247
...,...,...,...,...
558,559,07/29/15 09:30:00 AM,0.54884,14.194
559,560,07/29/15 09:45:00 AM,0.54884,14.242
560,561,07/29/15 10:00:00 AM,0.54884,14.194
561,562,07/29/15 10:15:00 AM,0.54884,14.625


la suite est à adapter

In [20]:
def nombre_de_valeurs_manquantes(capteur):
    
    nombre_de_valeurs_manquantes = capteur.isnull().sum()
    nombre_de_valeurs_manquantes[0:6] 


    return nombre_de_valeurs_manquantes

nombre_de_valeurs_manquantes_pression = nombre_de_valeurs_manquantes(capteur_pression)
print(nombre_de_valeurs_manquantes_pression)

nombre_de_valeurs_manquantes_temperature = nombre_de_valeurs_manquantes(capteur_temperatures)
print(nombre_de_valeurs_manquantes_temperature)

#                                                    0
Date Heure, GMT+02:00                                0
Tension, V (LGR S/N: 10117169, SEN S/N: 10117169)    0
Temp., °C (LGR S/N: 10117169, SEN S/N: 10117169)     0
dtype: int64
#                                                     0
Date Heure, GMT+02:00                                 0
Temp., °C (LGR S/N: 10117154, SEN S/N: 10117154)      0
Temp., °C (LGR S/N: 10117154, SEN S/N: 10117154).1    0
Temp., °C (LGR S/N: 10117154, SEN S/N: 10117154).2    0
Temp., °C (LGR S/N: 10117154, SEN S/N: 10117154).3    0
dtype: int64


In [70]:
#Supression de colonnes inutiles

if "#" in capteur_pression.columns:
    capteur_pression = capteur_pression.drop("#", axis=1)
    
if "#" in capteur_temperatures.columns:
    capteur_temperatures = capteur_temperatures.drop("#", axis=1)
